# BYOC training for paddleOCR

In [28]:
!cat Dockerfile

#FROM paddlepaddle/paddle:2.0.0rc1-gpu-cuda10.1-cudnn7#
FROM registry.baidubce.com/paddlepaddle/paddle:2.2.2-gpu-cuda10.2-cudnn7
#ARG REGISTRY_URI
#FROM ${REGISTRY_URI}/pytorch-training:1.3.1-cpu-py36-ubuntu16.04
#FROM ${REGISTRY_URI}/pytorch-training:1.5.1-gpu-py36-cu101-ubuntu16.04
#FROM ${REGISTRY_URI}/autogluon-training:0.2.1-gpu-py37-cu102-ubuntu18.04

ENV LANG=en_US.utf8
ENV LANG=C.UTF-8

ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE
ENV PATH="/opt/program:${PATH}"
#ENV LD_LIBRARY_PATH='/usr/local/cuda-10.1/targets/x86_64-linux/lib:${LD_LIBRARY_PATH:+:${LD_LIBRARY_PATH}}'

##########################################################################################
# SageMaker requirements
##########################################################################################
RUN pip3 install --upgrade pip

## install flask
#RUN pip3 install networkx==2.3 flask gevent gunicorn boto3 paddleocr>=2.0.1
#todo: need to test the paddleocr version
RUN pip3 install networkx=

In [29]:
#first build docker
!sh build_and_push.sh

set -e
# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.
# here we need to specify aksk

# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.
image=$1

if [ "$image" == "" ]
then
    echo "Use image name paddle"
    image="paddle"
fi
Use image name paddle

# Get the account number associated with the current IAM credentials
account=$(aws sts get-caller-identity --query Account --output text)

if [ $? -ne 0 ]
then
    exit 255
fi

# Get the region defined in the current configuration
#if you only want to specify one region, use region, otherwise, use regions
region=$(aws configure get region)

if [[ $region =~ ^cn.* ]]
then
d
elif [[ $region = "ap-east-1" ]]
then
    fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image}:latest"
    registry_id="871362719292"
    registry_uri="${registry_id}.dkr.ecr.

## step1: upload data

In [15]:
import sagemaker as sage
from time import gmtime, strftime
from sagemaker import get_execution_role

#sess = sage.Session()
sess = sage.LocalSession()

WORK_DIRECTORY = "./input/data"

# S3 prefix
prefix = "DEMO-paddle-byo"

role = get_execution_role()

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

## step2: local train (to debug)

In [16]:
# local mode
account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name
image = "847380964353.dkr.ecr.us-west-2.amazonaws.com/paddle"
sess = sage.LocalSession()

hyperparameters = {"epoch_num": 10,
                  "print_batch_step":5,
                  "save_epoch_step":30,
                  'pretrained_model':'/opt/program/pretrain/ch_ppocr_mobile_v2.0_rec_train/best_accuracy'}

train = sage.estimator.Estimator(
    image,
    role,
    instance_count = 1,
    sagemaker_session=sess,
    instance_type='local_gpu',
    hyperparameters=hyperparameters,
)


train.fit(data_location)

Creating kirocbnzzz-algo-1-0rg0k ... 
Creating kirocbnzzz-algo-1-0rg0k ... done
Attaching to kirocbnzzz-algo-1-0rg0k
kirocbnzzz-algo-1-0rg0k | Starting the training.
kirocbnzzz-algo-1-0rg0k | <<<< input files:  ['/opt/ml/input/data/training/rec_gt_train.txt', '/opt/ml/input/data/training/.ipynb_checkpoints', '/opt/ml/input/data/training/test', '/opt/ml/input/data/training/rec_gt_test.txt', '/opt/ml/input/data/training/train']
kirocbnzzz-algo-1-0rg0k | [2022/04/13 05:47:03] root INFO: train.py with paddle 2.2.2 and device CUDAPlace(0)
kirocbnzzz-algo-1-0rg0k | [2022/04/13 05:47:03] root INFO: Initialize indexs of datasets:['/opt/ml/input/data/training/rec_gt_train.txt']
kirocbnzzz-algo-1-0rg0k | [2022/04/13 05:47:03] root INFO: Initialize indexs of datasets:['/opt/ml/input/data/training/rec_gt_test.txt']
kirocbnzzz-algo-1-0rg0k | W0413 05:47:03.395622     1 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.0, Runtime API Version: 10.2
kir

## optional: create training job for training

In [17]:
account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name
image = "847380964353.dkr.ecr.us-west-2.amazonaws.com/paddle"

sess = sage.Session()
hyperparameters = {"epoch_num": 10,
                  "print_batch_step":5,
                  "save_epoch_step":3,
                  'pretrained_model':'/opt/program/pretrain/ch_ppocr_mobile_v2.0_rec_train/best_accuracy'}

train = sage.estimator.Estimator(
    image,
    role,
    instance_count = 1,
    sagemaker_session=sess,
    instance_type='ml.p3.2xlarge',
    hyperparameters=hyperparameters,
)


train.fit(data_location)

2022-04-13 05:47:20 Starting - Starting the training job...
2022-04-13 05:47:46 Starting - Preparing the instances for trainingProfilerReport-1649828840: InProgress
.........
2022-04-13 05:49:17 Downloading - Downloading input data
2022-04-13 05:49:17 Training - Downloading the training image..................
2022-04-13 05:52:13 Training - Training image download completed. Training in progress..Starting the training.
<<<< input files:  ['/opt/ml/input/data/training/.ipynb_checkpoints', '/opt/ml/input/data/training/rec_gt_train.txt', '/opt/ml/input/data/training/rec_gt_test.txt', '/opt/ml/input/data/training/train', '/opt/ml/input/data/training/test']
[2022/04/13 05:52:19] root INFO: train.py with paddle 2.2.2 and device CUDAPlace(0)
[2022/04/13 05:52:19] root INFO: Initialize indexs of datasets:['/opt/ml/input/data/training/rec_gt_train.txt']
[2022/04/13 05:52:19] root INFO: Initialize indexs of datasets:['/opt/ml/input/data/training/rec_gt_test.txt']
W0413 05:52:20.008754     1 devi

 # Inference

A trained model does nothing on its own. We now want to use the model to perform inference. For this example, that means predicting the topic mixture representing a given document.This section involves several steps,
Create Model 
- Create model for the training outputCreate Endpoint Configuration 
- Create a configuration defining an endpoint.Create Endpoint 
- Use the configuration to create an inference endpoint.Perform Inference 
- Perform inference on some input data using the endpoint.

## deploy model
we now create a SageMaker Model from the training output. Using the model we can create an Endpoint Configuration.


In [30]:
%%time

import boto3
from time import gmtime, strftime

sage = boto3.Session().client(service_name='sagemaker') 

from sagemaker import get_execution_role
role = get_execution_role()

model_name="paddle-v0"
print(model_name)

info = sage.describe_training_job(TrainingJobName="paddle-2022-04-13-05-47-20-163")
model_data = info['ModelArtifacts']['S3ModelArtifacts']

#model_data = train.model_data
#print(model_data)

hosting_image = "847380964353.dkr.ecr.us-west-2.amazonaws.com/paddle"
primary_container = {
    'Image': hosting_image,
    'ModelDataUrl': model_data,
}

create_model_response = sage.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response['ModelArn'])

paddle-v0
arn:aws:sagemaker:us-west-2:847380964353:model/paddle-v0
CPU times: user 132 ms, sys: 16 ms, total: 148 ms
Wall time: 490 ms


## Create Endpoint Configuration
#At launch, we will support configuring REST endpoints in hosting with multiple models, e.g. for A/B testing purposes. In order to support this, customers create an endpoint configuration, that describes the distribution of traffic across the models, whether split, shadowed, or sampled in some way.In addition, the endpoint configuration describes the instance type required for model deployment, and at launch will describe the autoscaling configuration.


In [31]:
from time import gmtime, strftime
import time 

job_name_prefix = "paddle"

timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_config_name = job_name_prefix + '-epc-' + timestamp
endpoint_config_response = sage.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.g4dn.xlarge',
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print('Endpoint configuration name: {}'.format(endpoint_config_name))
print('Endpoint configuration arn:  {}'.format(endpoint_config_response['EndpointConfigArn']))

Endpoint configuration name: paddle-epc--2022-04-13-06-03-09
Endpoint configuration arn:  arn:aws:sagemaker:us-west-2:847380964353:endpoint-config/paddle-epc--2022-04-13-06-03-09


##  Create Endpoint
Lastly, the customer creates the endpoint that serves up the model, through specifying the name and configuration defined above. The end result is an endpoint that can be validated and incorporated into production applications. This takes 9-11 minutes to complete.

In [32]:
%%time
import time

timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_name = job_name_prefix + '-ep-' + timestamp
print('Endpoint name: {}'.format(endpoint_name))

endpoint_params = {
    'EndpointName': endpoint_name,
    'EndpointConfigName': endpoint_config_name,
}
endpoint_response = sage.create_endpoint(**endpoint_params)
print('EndpointArn = {}'.format(endpoint_response['EndpointArn']))

Endpoint name: paddle-ep--2022-04-13-06-03-10
EndpointArn = arn:aws:sagemaker:us-west-2:847380964353:endpoint/paddle-ep--2022-04-13-06-03-10
CPU times: user 6.31 ms, sys: 0 ns, total: 6.31 ms
Wall time: 247 ms


 If you see the message, Endpoint creation ended with ```EndpointStatus = InService``` then congratulations! You now have a functioning inference endpoint. You can confirm the endpoint configuration and status by navigating to the "Endpoints" tab in the AWS SageMaker console.We will finally create a runtime object from which we can invoke the endpoint.

#  Perform Inference

In [37]:
%%time
import boto3
from botocore.config import Config
from sagemaker.session import Session

config = Config(
    read_timeout=120,
    retries={
        'max_attempts': 0
    }
)

from boto3.session import Session
import json

#bucket = 'datalab2022'
#image_uri = '1.jpg'
bucket = 'bot-ocr-test-bucket'
image_uri = '1.png'
test_data = {
    'bucket' : bucket,
    'image_uri' : image_uri,
    'content_type': "application/json",
}
payload = json.dumps(test_data)
print(payload)

sagemaker_runtime_client = boto3.client('sagemaker-runtime', config=config)
session = Session(sagemaker_runtime_client)

#     runtime = session.client("runtime.sagemaker",config=config)
response = sagemaker_runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=payload)

result = json.loads(response["Body"].read())
print (result)

{"bucket": "bot-ocr-test-bucket", "image_uri": "1.png", "content_type": "application/json"}
{'label': ['婴巴马', '别男', '糖肯尼亚', '出些·1961年4日', '华盛顿特区宜宾法尼亚', '大道1600号白宫', '公E与盛', '12345196108047&9③'], 'confidences': [12.665797233581543, 10.436362266540527, 12.495158195495605, 13.70422077178955, 16.082889556884766, 14.658669471740723, 9.809164047241211, 15.068533897399902], 'bbox': [[[32.0, 64.0], [241.0, 56.0], [243.0, 92.0], [33.0, 100.0]], [[34.0, 138.0], [179.0, 138.0], [179.0, 173.0], [34.0, 173.0]], [[250.0, 135.0], [435.0, 134.0], [436.0, 165.0], [250.0, 166.0]], [[32.0, 212.0], [462.0, 209.0], [462.0, 246.0], [33.0, 249.0]], [[28.0, 293.0], [505.0, 291.0], [506.0, 332.0], [28.0, 334.0]], [[131.0, 365.0], [396.0, 362.0], [396.0, 396.0], [131.0, 398.0]], [[30.0, 494.0], [246.0, 494.0], [246.0, 528.0], [30.0, 528.0]], [[309.0, 496.0], [791.0, 492.0], [791.0, 526.0], [309.0, 530.0]]], 'shape': [588, 940, 3]}
CPU times: user 29.7 ms, sys: 0 ns, total: 29.7 ms
Wall time: 756 ms


# Clean up
When we're done with the endpoint, we can just delete it and the backing instances will be released.  Run the following cell to delete the endpoint.

In [27]:
print(endpoint_name)
sage.delete_endpoint(EndpointName=endpoint_name)
sage.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sage.delete_model(ModelName=model_name)

paddle-ep--2022-04-13-05-54-10


{'ResponseMetadata': {'RequestId': '842e4591-7ecf-42bb-950e-8d9cb273a478',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '842e4591-7ecf-42bb-950e-8d9cb273a478',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 13 Apr 2022 06:02:27 GMT'},
  'RetryAttempts': 0}}

In [24]:
endpoint_name

'paddle-ep--2022-04-13-05-54-10'